In [29]:
from itertools import product, combinations_with_replacement
from operator import itemgetter, add, mul, sub, truediv, pow
from sympy import srepr, sympify, symbols, exp, sin, cos, ln, asin, acos
from math import factorial
x,c = symbols('x c')

In [13]:
all_nodes = {
    'x': {
        'children': 0,
        'op': x
    },
    'c': {
        'children': 0,
        'op': c # Konstante
    },
    'exp': {
        'children': 1,
        'op': exp
    },
    'ln': {
        'children': 1,
        'op': ln
    },
    'sin': {
        'children': 1,
        'op': sin
    },
    'cos': {
        'children': 1,
        'op': cos
    },
    'asin': {
        'children': 1,
        'op': asin
    },
    'acos': {
        'children': 1,
        'op': acos
    },
    '+': {
        'children': 2,
        'op': add,
        'commutative': True
    },
    '-': {
        'children': 2,
        'op': sub,
        'commutative': False
    },
    '*': {
        'children': 2,
        'op': mul,
        'commutative': True
    },
    '/': {
        'children': 2,
        'op': truediv,
        'commutative': False
    },
    # '**': {
    #     'children': 2,
    #     'op': pow,
    #     'commutative': False
    # }
}

In [6]:
def generate_expressions(sub_expressions):
    for properties in all_nodes.values():
        children, operation = itemgetter('children', 'op')(properties)
        if children == 2 and properties['commutative'] == True:
            for left, right in combinations_with_replacement(sub_expressions, 2):
                yield operation(sympify(left), sympify(right))
        elif children == 2:
            for left, right in product(sub_expressions, repeat=2):
                yield operation(sympify(left), sympify(right))
        elif children == 1:
            for expr in sub_expressions:
                yield operation(sympify(expr))

In [7]:
def save_expressions(depth):
    uniques = set()
    if depth <= 1:
        uniques = set([x, c])
    else:
        with open(f'uniques_ext_depth{depth - 1}.csv', 'r') as file:
            for line in file:
                uniques.add(line)

    with open(f'expressions_ext_depth{depth}.csv', 'w') as file:
        for expr in generate_expressions(uniques):
            file.write(srepr(expr))
            file.write('\n')

In [8]:
def remove_unique_expressions(depth):
    uniques = set()
    with open(f'expressions_ext_depth{depth}.csv', 'r') as file:
        for line in file:
            uniques.add(line)
    with open(f'uniques_ext_depth{depth}.csv', 'w') as file:
        for line in uniques:
            file.write(line)

In [26]:
save_expressions(2)

In [27]:
remove_unique_expressions(2)

In [31]:
def calculate_complexity(n_expr):
    n_commutative = len([n for n in all_nodes.values() if 'commutative' in n and n['commutative']])
    n_dual = len([n for n in all_nodes.values() if 'commutative' in n and not n['commutative'] and n['children'] == 2])
    n_single = len([n for n in all_nodes.values() if n['children'] == 1])
    commutative_combinations = factorial(n_expr+1) / 2 / factorial(n_expr-1)
    return n_commutative * commutative_combinations + n_dual * n_expr**2 + n_single * n_expr